In [27]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
import time
from tqdm.notebook import tqdm
import datetime
import os
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from sklearn.metrics import mean_squared_error

import scipy.sparse.linalg
from scipy.sparse import coo_matrix

In [17]:
train_joke_df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')

In [5]:
joke_df = pd.read_csv(r'..\joke_df_features_svd.csv')
user_df = pd.read_csv(r'..\user_df_features_svd.csv')

In [6]:
joke_df

,joke_feature_1,joke_feature_2,joke_feature_3,joke_feature_4,joke_feature_5,joke_feature_6,joke_feature_7,joke_feature_8,joke_feature_9,joke_feature_10,...,joke_feature_91,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100
0,0.112688,0.222817,-0.195922,0.219684,0.411241,-0.053139,0.290816,-0.058500,0.003905,0.066091,...,0.431792,-0.320441,0.025341,0.102580,-0.387499,0.224376,0.364185,-0.144255,-0.240137,-0.015587
1,0.313724,0.043327,-0.048844,0.067215,0.307983,0.014399,0.256734,-0.074591,-0.023945,0.102127,...,0.623640,0.011707,-0.018323,-0.022698,-0.248734,-0.178312,0.374911,-0.056900,-0.330262,-0.087538
2,-0.252002,0.430994,0.220400,0.133130,0.135608,0.111379,0.129852,0.039378,-0.162345,0.195971,...,-0.162176,-0.297996,-0.132594,0.337949,0.067280,0.285934,-0.153183,0.132184,0.000751,-0.057890
3,-0.052535,0.541485,-0.223866,0.139070,0.385290,0.131492,0.507533,0.002464,-0.133843,-0.067574,...,0.040186,-0.182518,0.084169,0.219612,-0.213434,0.361682,0.079440,-0.318766,-0.021421,0.146524
4,-0.027378,0.174890,-0.053249,0.102202,0.344133,0.069124,0.256610,-0.168126,-0.235313,0.104072,...,-0.037136,-0.286614,-0.147054,0.241144,0.003931,0.260650,0.021384,-0.104395,0.103104,0.084330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.137697,0.603971,-0.180179,-0.214594,0.265756,0.238847,0.676550,0.098790,-0.127231,0.063033,...,0.437792,0.151213,0.410687,0.035605,-0.268979,0.302526,0.168139,-0.008251,-0.173199,0.090621
96,-0.005569,0.412936,-0.002250,-0.214248,0.468186,0.145344,0.627553,-0.441548,0.129704,0.086022,...,0.078341,-0.180102,-0.107962,0.078101,0.119086,0.374430,0.161647,-0.568151,0.198361,0.353774
97,0.178174,-0.115281,-0.007642,-0.393356,0.423065,0.120142,0.585237,-0.422318,0.128890,0.172139,...,0.714508,-0.029828,0.136986,-0.066974,-0.150649,-0.105737,0.449684,-0.618395,0.196442,0.112897
98,-0.338073,0.425045,-0.089250,-0.360074,0.323095,0.378184,0.690955,-0.020013,-0.026818,-0.028049,...,0.369752,-0.075770,0.261672,0.101164,-0.216765,0.316440,0.162326,-0.422756,0.159005,0.225908


In [8]:
test_joke_df_nofactrating = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)
test_joke_df_nofactrating

,UID,JID
InteractionID,,
0,11228,39
1,21724,85
2,16782,56
3,12105,42
4,14427,2
...,...,...
362086,3085,66
362087,13765,31
362088,10341,29


In [20]:
train_joke_df

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60
...,...,...,...
1448359,22604,26,2.82
1448360,22255,36,-1.94
1448361,21056,40,-9.56
1448362,12328,97,0.87


In [21]:
ratings = []
for ind, row in enumerate(tqdm(train_joke_df.values)):
    u = user_df.iloc[int(row[0])-1]
    i = joke_df.iloc[int(row[1])-1]
    rating = u.values @ i.values
    ratings.append(rating)
    
        
        
train_joke_df['rating_predict'] = ratings

In [22]:
train_joke_df

,UID,JID,Rating,rating_predict
0,18029,6,-1.26,-0.345800
1,3298,64,-4.17,1.611948
2,3366,58,0.92,-0.413655
3,12735,92,3.69,1.737163
4,11365,38,-6.60,-1.295192
...,...,...,...,...
1448359,22604,26,2.82,-1.138970
1448360,22255,36,-1.94,0.506446
1448361,21056,40,-9.56,-1.395329
1448362,12328,97,0.87,0.630667


In [25]:
mean_squared_error(train_joke_df.Rating, train_joke_df.rating_predict, squared=False)

5.794805684224242

In [12]:
ratings = []
for row in test_joke_df_nofactrating.vaules:
    u = user_df.iloc[int(row.UID)-1]
    i = joke_df.iloc[int(row.JID)-1]
    
    rating = u.values @ i.values
    
    ratings.append(rating)

user_feature_1      0.094999
user_feature_2     -0.354062
user_feature_3      0.049003
user_feature_4      0.232890
user_feature_5     -0.000136
                      ...   
user_feature_96    -0.025408
user_feature_97    -0.566886
user_feature_98     0.158692
user_feature_99    -0.335032
user_feature_100   -0.374157
Name: 11227, Length: 100, dtype: float64

In [13]:
i = joke_df.iloc[39-1]
i

joke_feature_1     -0.082742
joke_feature_2      0.510615
joke_feature_3     -0.016505
joke_feature_4     -0.242814
joke_feature_5      0.473243
                      ...   
joke_feature_96     0.394105
joke_feature_97     0.067036
joke_feature_98    -0.631281
joke_feature_99     0.274659
joke_feature_100    0.389250
Name: 38, Length: 100, dtype: float64

-1.3906942954161283

-1.3906942954161283

In [30]:
row_list = []
col_list = []
data_list = []


for row in train_joke_df.values:
    user_id = int(row[0])
    joke_id = int(row[1])
    rating = row[2]
    
    row_list.append(user_id)
    col_list.append(joke_id)
    data_list.append(rating)
    
    

row = np.array(row_list)
col = np.array(col_list)
data = np.array(data_list)

In [31]:
coo = coo_matrix((data, (row, col)), shape=(np.max(row)+1, np.max(col)+1))

In [35]:
coo_ = coo.toarray()

In [36]:
u, s, vh = np.linalg.svd(coo_, full_matrices=False)

u.shape, s.shape, vh.shape

((24984, 101), (101,), (101, 101))

In [37]:
rec = u @ np.diag(s) @ vh

In [78]:
rec.shape

(24984, 101)

In [75]:
train_joke_df

,UID,JID,Rating,rating_predict
0,18029,6,-1.26,-1.26
1,3298,64,-4.17,-4.17
2,3366,58,0.92,0.92
3,12735,92,3.69,3.69
4,11365,38,-6.60,-6.60
...,...,...,...,...
1448359,22604,26,2.82,2.82
1448360,22255,36,-1.94,-1.94
1448361,21056,40,-9.56,-9.56
1448362,12328,97,0.87,0.87


In [49]:
coo_[18029, 6]

-1.26

In [50]:
rec[18029, 6]

-1.2599999999999936

In [73]:
(u[18029] * s[6]) @ vh[6]

-3.505906730879314

In [71]:
(u @ np.diag(s))[18029] @ vh[5]

-3.6873455705976563

In [80]:
uu = u @ np.diag(s)

uu.shape

(24984, 101)

In [81]:
rec2 = uu @ vh

In [38]:
np.linalg.norm(rec - coo_)

1.8514504485465976e-11

In [82]:
ratings = []
for ind, row in enumerate(tqdm(train_joke_df.values)):
    rating = rec2[int(row[0]), int(row[1])]
    ratings.append(rating)
    
train_joke_df['rating_predict'] = ratings

mean_squared_error(train_joke_df.Rating, train_joke_df.rating_predict, squared=False)

  0%|          | 0/1448364 [00:00<?, ?it/s]

1.2698478763133556e-14

In [54]:
ratings = []
for ind, row in enumerate(tqdm(test_joke_df_nofactrating.values)):
    rating = rec[int(row[0]), int(row[1])]
    ratings.append(rating)
    
test_joke_df_nofactrating['Rating'] = ratings
test_joke_df_nofactrating

  0%|          | 0/362091 [00:00<?, ?it/s]

,UID,JID,Rating
InteractionID,,,
0,11228,39,1.164125e-14
1,21724,85,1.226154e-14
2,16782,56,-3.460948e-15
3,12105,42,-3.147068e-14
4,14427,2,1.399569e-14
...,...,...,...
362086,3085,66,-7.645501e-15
362087,13765,31,-4.031407e-16
362088,10341,29,-4.270601e-15


In [77]:
test_joke_df_nofactrating[np.abs(test_joke_df_nofactrating.Rating) > 0.1]

,UID,JID,Rating
InteractionID,,,
